In [ ]:
import pandas as pd
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.mask import mask
from rasterio.features import geometry_mask
import numpy as np
import matplotlib.dates as mdates

In [ ]:
counties = gpd.read_file()
path = ''
points_df = pd.read_csv(path, parse_dates=['date'])
if 'longitude' not in points_df.columns or 'latitude' not in points_df.columns:
    raise ValueError(" must contain coors")
geometry = [Point(xy) for xy in zip(points_df['longitude'], points_df['latitude'])]
points = gpd.GeoDataFrame(points_df, geometry=geometry)
points.set_crs(epsg=4326, inplace=True)
counties = counties.to_crs(points.crs)
points_with_counties = gpd.sjoin(points, counties, how='left', predicate='within')
story_county_points = points_with_counties[points_with_counties['CountyName'] == 'Story']

if story_county_points.empty:
    raise ValueError("No data found for Story County.")

story_coords = story_county_points[['latitude', 'longitude']].drop_duplicates()
print(story_coords)


In [ ]:
path = 
data = pd.read_csv(path, parse_dates=['date'])
data = data.set_index('date')

In [ ]:
data['soil_moisture'] = pd.to_numeric(data['soil_moisture'])
ldata = data[(data['latitude'] == 41.948936) & (data['longitude'] == -93.687760)]
print(ldata.head())
#style = mpf.make_mpf_style(base_mpl_style = 'default', rc={'font.size': 8 })

#fig, axes = mpf.plot(data, type='candle', volume=True, title='Candle stick 3hr resolution', ylabel = 'Soil Moisture', show_nontrading=True, returnfig=True, style=style)
#plt.style.use('dark_background')

In [ ]:
ldata['Open'] = ldata['soil_moisture'].shift(1)
ldata['Close'] = ldata['soil_moisture']
ldata['High'] = ldata['soil_moisture']
ldata['Low'] = ldata['soil_moisture']
ldata.dropna(inplace=True)

mpf.plot(ldata, type='candle', style='charles', title=f'Candlestick Plot for SM ', ylabel='Soil Moisture', volume=False)
plt.show()

In [ ]:
sixhrdata = ldata.iloc[::2]
#intermediate_vals = ldata.iloc[1::2]
#nonvals = ldata.iloc[1::2]
print(sixhrdata.head())
sixhrdata['Open'] = sixhrdata['soil_moisture'].shift(1)
sixhrdata['Close'] = sixhrdata['soil_moisture']
sixhrdata['High'] = sixhrdata['soil_moisture']
sixhrdata['Low'] = sixhrdata['soil_moisture']

sixhrdata.dropna(inplace=True)

#Figure out how to add wicks given that the mpf fiannce modoule only takes 1 input dataframe

mpf.plot(sixhrdata, type='candle', style='charles', title=f'Candlestick Plot for 6hr Aggegated SM ', ylabel='Soil Moisture', volume=False)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf


new_length = len(ldata) - (len(ldata) % 4)
ldata_new = ldata.iloc[:new_length]


open_values = []
high_values = []
low_values = []
close_values = []
volume_values = []
index_values = []

# Iteratewith a step size of 4 (representing 12-hour intervals)
for i in range(0, len(ldata_new) - 4, 4):
    # get the data for the current 12-hour period
    period_data = ldata_new['soil_moisture'].iloc[i:i+4]
    
    # Open, High, Low, Close, and Volume
    open_value = period_data.iloc[0]          # First value in the 12-hour period
    high_value = period_data.max()            # Maximum value in the 12-hour period
    low_value = period_data.min()             # Minimum value in the 12-hour period
    close_value = ldata_new['soil_moisture'].iloc[i+4]  # open of the next period as the close of this period
    volume_value = np.var(period_data)          # variance within the period as volume
    
    open_values.append(open_value)
    high_values.append(high_value)
    low_values.append(low_value)
    close_values.append(close_value)
    volume_values.append(volume_value)
    index_values.append(ldata_new.index[i])

    print(f"Interval {i//4 + 1}:")
    print(f"  Open: {open_value}")
    print(f"  High: {high_value}")
    print(f"  Low: {low_value}")
    print(f"  Close: {close_value}")
    print(f"  Volume (Variance): {volume_value}")
    print()

df_resampled = pd.DataFrame({
    'Open': open_values,
    'High': high_values,
    'Low': low_values,
    'Close': close_values,
    'Volume': volume_values
}, index=index_values)

df_resampled.dropna(inplace=True)
print(df_resampled.head(10))

mpf.plot(df_resampled, type='candle', style='charles', title='12-Hour Candlestick Chart with Variance as Volume', ylabel='Soil Moisture', volume=True)
plt.show()


In [ ]:
new_length = len(ldata) - (len(ldata) % 8)
ldata_new = ldata.iloc[:new_length]

open_values = []
high_values = []
low_values = []
close_values = []
volume_values = []
index_values = []

for i in range(0, len(ldata_new) - 8, 8):
    period_data = ldata_new['soil_moisture'].iloc[i:i+8]
    
    open_value = period_data.iloc[0]
    high_value = period_data.max()
    low_value = period_data.min()
    close_value = ldata_new['soil_moisture'].iloc[i+8]
    volume_value = np.var(period_data)
    
    open_values.append(open_value)
    high_values.append(high_value)
    low_values.append(low_value)
    close_values.append(close_value)
    volume_values.append(volume_value)
    
    index_values.append(ldata_new.index[i])

    print(f"Interval {i//8 + 1}:")
    print(f"  Actual values: {period_data.values}")
    print(f"  Open: {open_value}")
    print(f"  High: {high_value}")
    print(f"  Low: {low_value}")
    print(f"  Close: {close_value}")
    print(f"  Volume (Variance): {volume_value}")
    print()

df_resampled = pd.DataFrame({
    'Open': open_values,
    'High': high_values,
    'Low': low_values,
    'Close': close_values,
    'Volume': volume_values
}, index=index_values)

df_resampled.dropna(inplace=True)

print("\nFinal DataFrame for Candlestick Plotting with Volume:")
print(df_resampled.head(10))

mpf.plot(df_resampled, type='candle', style='charles', title='24-Hour Candlestick Chart with Variance as Volume', ylabel='Soil Moisture', volume=True)
plt.show()


In [ ]:
new_length = len(ldata) - (len(ldata) % 16)
ldata_new = ldata.iloc[:new_length]

open_values = []
high_values = []
low_values = []
close_values = []
volume_values = []
index_values = []

for i in range(0, len(ldata_new) - 16, 16):
    period_data = ldata_new['soil_moisture'].iloc[i:i+16]
    
    open_value = period_data.iloc[0]
    high_value = period_data.max()
    low_value = period_data.min()
    close_value = ldata_new['soil_moisture'].iloc[i+16]
    volume_value = np.var(period_data)
    
    open_values.append(open_value)
    high_values.append(high_value)
    low_values.append(low_value)
    close_values.append(close_value)
    volume_values.append(volume_value)
    
    index_values.append(ldata_new.index[i])

    print(f"Interval {i//16 + 1}:")
    print(f"  Actual values: {period_data.values}")
    print(f"  Open: {open_value}")
    print(f"  High: {high_value}")
    print(f"  Low: {low_value}")
    print(f"  Close: {close_value}")
    print(f"  Volume (Variance): {volume_value}")
    print()

df_resampled = pd.DataFrame({
    'Open': open_values,
    'High': high_values,
    'Low': low_values,
    'Close': close_values,
    'Volume': volume_values
}, index=index_values)

df_resampled.dropna(inplace=True)

print("\nFinal DataFrame for Candlestick Plotting with Volume:")
print(df_resampled.head(10))

mpf.plot(df_resampled, type='candle', style='charles', title='48-Hour Candlestick Chart with Variance as Volume', ylabel='Soil Moisture', volume=True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf

new_length = len(ldata) - (len(ldata) % 32)
ldata_new = ldata.iloc[:new_length]

open_values = []
high_values = []
low_values = []
close_values = []
volume_values = []
index_values = []

for i in range(0, len(ldata_new) - 32, 32):
    period_data = ldata_new['soil_moisture'].iloc[i:i+32]
    
    open_value = period_data.iloc[0]
    high_value = period_data.max()
    low_value = period_data.min()
    close_value = ldata_new['soil_moisture'].iloc[i+32]
    volume_value = np.var(period_data)
    
    open_values.append(open_value)
    high_values.append(high_value)
    low_values.append(low_value)
    close_values.append(close_value)
    volume_values.append(volume_value)
    
    index_values.append(ldata_new.index[i])

    print(f"Interval {i//32 + 1}:")
    print(f"  Actual values: {period_data.values}")
    print(f"  Open: {open_value}")
    print(f"  High: {high_value}")
    print(f"  Low: {low_value}")
    print(f"  Close: {close_value}")
    print(f"  Volume (Variance): {volume_value}")
    print()

df_resampled = pd.DataFrame({
    'Open': open_values,
    'High': high_values,
    'Low': low_values,
    'Close': close_values,
    'Volume': volume_values
}, index=index_values)

df_resampled.dropna(inplace=True)

print("\nFinal DataFrame for Candlestick Plotting with Volume:")
print(df_resampled.head(10))

mpf.plot(df_resampled, type='candle', style='charles', title='96-Hour Candlestick Chart with Variance as Volume', ylabel='Soil Moisture', volume=True)
plt.show()


In [ ]:
#checking that it wokrs
values = [0.294888,0.293671]
variance = np.var(values)
print(f"variance: {variance}")


In [ ]:
dailydata = ldata.iloc[::8]
print(dailydata.head())
dailydata['Open'] = dailydata['soil_moisture'].shift(1)
dailydata['Close'] = dailydata['soil_moisture']
dailydata['High'] = dailydata['soil_moisture']
dailydata['Low'] = dailydata['soil_moisture']
dailydata.dropna(inplace=True)

mpf.plot(dailydata, type='candle', style='charles', title=f'Daily Candlestick Plot for Daily Aggegated SM ', ylabel='Soil Moisture', volume=False)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.dates as mdates

def create_candleStick(data, interval, title):
    selected_data = data.iloc[::interval].copy()
    selected_data['Open'] = selected_data['soil_moisture'].shift(1)
    selected_data['Close'] = selected_data['soil_moisture']
    selected_data['High'] = selected_data['soil_moisture']
    selected_data['Low'] = selected_data['soil_moisture']

    selected_data.dropna(inplace=True)
    fig, ax = plt.subplots(figsize=(18, 10))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax.xaxis.set_minor_locator(mdates.HourLocator(interval=12))
    ax.grid(True, which='major', axis='x', color='black', linestyle='-', linewidth=0.5)
    ax.grid(True, which='minor', axis='x', color='lightgray', linestyle='--', linewidth=0.5)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    mpf.plot(selected_data, type='candle', style='charles', ax=ax, ylabel='Volumetric Soil Moisture', volume=False)
    ax.set_title(f'Candlestick Plot for Soil Moisture ({title})')

    plt.tight_layout()
    plt.show()


path = 
data = pd.read_csv(path, parse_dates=['date'])
data.set_index('date', inplace=True)
data['soil_moisture'] = pd.to_numeric(data['soil_moisture'])


ldata = data[(data['latitude'] == 41.948936) & (data['longitude'] == -93.687760)]

create_candleStick(ldata, 2, '6-Hour Interval')
create_candleStick(ldata, 3, '9-Hour Interval')
create_candleStick(ldata, 4, '12-Hour Interval')
create_candleStick(ldata, 8, '24-Hour Interval')


In [ ]:

cdata =data[(data['latitude'] == 41.948936) & (data['longitude'] == -93.687760)]

def calculate_non_overlapping_variance(data, window_size):
    variances = []
    timestamps = []
    for i in range(0, len(data), window_size):
        window_data = data[i:i + window_size]
        if len(window_data) == window_size:  
            variances.append(window_data.var())
            timestamps.append(window_data.index[0])  
    return timestamps, variances

window_sizes = [2, 3, 4, 8]  
variance_dict = {}

for window_size in window_sizes:
    timestamps, variances = calculate_non_overlapping_variance(cdata['soil_moisture'], window_size)
    variance_dict[f'window{window_size}'] = (timestamps, variances)

plt.figure(figsize=(14, 7))

for window_size in window_sizes:
    timestamps, variances = variance_dict[f'window {window_size}']
    plt.plot(timestamps, variances, label=f'Aggregation: {window_size*3}hr')

plt.xlabel('Date')
plt.ylabel('Variance')
plt.title('Variance for Different Aggregations')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

for window_size in window_sizes:
    print(f'variance for  {window_size}:')
    timestamps, variances = variance_dict[f'window_{window_size}']
    for ts, var in zip(timestamps, variances):
        print(f'Date: {ts}, Variance: {var}')
    print()


In [ ]:
path = ''
data = pd.read_csv(path, parse_dates=['date'])
data = data.set_index('date')

data['soil_moisture'] = pd.to_numeric(data['soil_moisture'])
cdata =data[(data['latitude'] == 41.948936) & (data['longitude'] == -93.687760)]

def calculate_non_overlapping_variance(cdata, window_size):
    variances = []
    timestamps = []
    for i in range(0, len(cdata), window_size):
        window_data = cdata.iloc[i:i + window_size]
        if len(window_data) == window_size:  
            variances.append(window_data.var())
            timestamps.append(window_data.index[0])  
    return timestamps, variances

window_sizes = [2, 3, 4, 8]
plt.figure(figsize=(14, 7))

for window_size in window_sizes:
    timestamps, variances = calculate_non_overlapping_variance(cdata['soil_moisture'], window_size)
    plt.plot(timestamps, variances, marker='o', label=f'{window_size*3}hr Window')

plt.xlabel('Date')
plt.ylabel('Variance')
plt.title('Variance at Different Resolutions')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
